# Four Factors

Para comenzar con este script, primero hay que aclarar que se van a calcular ocho factores.  
Esto se debe al procedimiento de cálculo de victorias: primero se estima la puntuación de cada partido, y luego se compara con el del rival. Debido a esto, hace falta "predecir" la puntuación del rival y, con lo cual, calcular sus four factors primero.

In [1]:
# Primero se importan las librerías
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# A continuación se cargan los datos
wdir = os.path.dirname(os.path.realpath('__file__'))
data = pd.read_csv(wdir+'/fourFactors.csv')

Ahora hace falta declarar lo que son los "ejes" de la función. Se sabe que los cuatro factores tienen un peso DESCONOCIDO, un coeficiente. Por este motivo, van al eje X, y con lo cual la puntuación final va al eje Y.

In [3]:
# Primero las del equipo deseado
x = data[['efg', 'ftr', 'orb', 'tov']]
y = data['scr']

# A continuación las del equipo rival
x_opp = data[['efg_opp', 'ftr_opp', 'orb_opp', 'tov_opp']]
y_opp = data['scr_opp']

Ahora se necesita separar los datos, puesto que no se pueden usar todos. Se separan unos datos de entrenamiento y otros de prueba para su uso posterior.

In [4]:
# Se genera una semilla de aleatoriedad (la misma para el equipo deseado y el rival, para que se separen los mismos partidos)
random = 7
# Se separan los datos
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=random)
x_train_opp, x_test_opp, y_train_opp, y_test_opp = train_test_split(x_opp, y_opp, random_state=random)

A continuación se inicia el algoritmo de regresión linear para obtener los coeficientes de cada factor.

In [5]:
# Inicio del algoritmo
estimate = LinearRegression()
estimate_opp = LinearRegression()

In [6]:
# Entrenamiento del modelo
estimate.fit(x_train, y_train)
estimate_opp.fit(x_train_opp, y_train_opp)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

### Obtención de los pesos

Puesto que ya se ha entrenado el modelo, se obtienen los coeficientes de cada factor para saber su importancia a la hora de obtener más puntos y, por consiguiente, la victoria.

In [7]:
print(estimate.coef_)
print(estimate_opp.coef_)

[129.33633385   7.5309764   24.56793731 -71.70886892]
[ 130.69376247    9.88463681   45.31953649 -105.92583942]


El primer valor se corresponde con el eFG%, el segundo con el TOV%, el tercero con el ORB% y el último con el FTR.

### Predicción de resultados
Como extensión de estos coeficientes, se puede tratar de predecir las victorias del equipo. Por este motivo, se usarán los datos que se dejaron fuera del entrenamiento para comprobarlo.

In [8]:
# Primero se hace una predicción de las puntuaciones del equipo deseado
y_pred = estimate.predict(x_test)
# Y se sigue con el rival
y_pred_opp = estimate.predict(x_test_opp)

In [9]:
# Debido a los formatos de NumPy al trabajar con SciKit-Learn, el tipo de variable es int64, el cual no es iterable.
# Hace falta convertirlo a una lista para poder trabajar con él.
p_results = y_pred.tolist()
p_results_opp = y_pred_opp.tolist()
results = y_test.tolist()
results_opp = y_test_opp.tolist()

In [10]:
# Se inician las variables para almacenar los resultados
p_won = []
won = []

Finalmente se hace un bucle para comparar los resultados predecidos con los reales, y así comprobar la eficacia del modelo. La variable `p_results` contiene los resultados predecidos del equipo deseado, `p_results_opp` es la puntuación del oponente, `results` es el resultado real del equipo deseado y `results_opp` es el resultado real del oponente.

In [11]:
counter = 0
while counter < len(p_results):
    p_won.append(1 if p_results[counter] > p_results_opp[counter] else 0)
    won.append(1 if results[counter] > results_opp[counter] else 0)
    counter += 1

In [12]:
# Se muestran los resultados
print(p_won)
print(won)

[1, 0, 0, 0, 0, 1, 0, 1, 0]
[1, 0, 0, 0, 0, 1, 0, 1, 1]


En la fila de arriba se ecuentran los resultados predecidos y en la de abajo están los reales. Tan solo falla un único partido, con lo cual se podría decir que el modelo funciona y es bastante fiable.